<a href="https://colab.research.google.com/github/BoredDany/ML-Python/blob/main/Chatbot_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot example using Gradio and MobileNetV2 (tensorflow)

By:

*   Daniela Martínez Quiroga
*   María Isabella Rodríguez Arévalo


In [ ]:
!pip install gradio

1. Creación de ChatBox

In [ ]:
def conversion(num, dir_check):
  try:
    if num < 0:
      raise ValueError
    elif dir_check:
      result = 3.281 * float(num)
      return "El resutado es: "+ str(result)+" pies"
    elif not dir_check:
      result = float(num)/3.281
      return "El resutado es: "+ str(result)+" metros"
  except ValueError:
    return "Enter a positive numeric value"


In [ ]:
import gradio as gr

#interfaz = gr.Interface(fn=metros_pies, inputs="number", outputs="textbox")

with gr.Blocks(css=".gradio-container {background-color: #f99abc;}") as interfaz:
  gr.Markdown("<h1 style='text-align: center; color: #d63384;'> Conversor de Metros a Pies </h1>")

  with gr.Column():
    input = [gr.Number(label="Ingrese la cantidad"),
             gr.Checkbox(label = "Dirección de la conversión (Check = metros a pie, Uncheck = pies a metros)")]
    btn = gr.Button("Convertir")
    output = gr.Textbox(label="Resultado")

  btn.click(conversion, inputs=input, outputs=output)

if __name__ == "__main__":
  interfaz.launch(share=True)

2. Chatbot para clasificar imágenes

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from PIL import Image

# Cargar el modelo preentrenado MobileNetV2
model = MobileNetV2(weights="imagenet")

def classify_image(image):
    """Clasifica la imagen usando el modelo MobileNetV2."""
    # Redimensionar la imagen al tamaño esperado por el modelo (224x224)
    image = image.resize((224, 224))

    # Convertir la imagen a un array de numpy y preprocesarla
    image_array = np.array(image)
    image_array = preprocess_input(image_array)
    image_array = np.expand_dims(image_array, axis=0)  # Añadir dimensión del batch

    # Realizar la predicción
    predictions = model.predict(image_array)

    # Decodificar las predicciones a etiquetas legibles
    decoded_predictions = decode_predictions(predictions, top=3)[0]  # Top 3 predicciones
    results = {label: float(confidence) for (_, label, confidence) in decoded_predictions}

    return results

# Crear la interfaz de Gradio
with gr.Blocks(css=".gradio-container {background-color: #f99abc;}") as app:
    gr.Markdown("<h1 style='text-align: center; color: #d63384;'> Clasificación de Imágenes con MobileNetV2 </h1>")

    # Componente para subir la imagen
    image_input = gr.Image(label="Subir Imagen", type="pil")

    # Botón para clasificar la imagen
    classify_button = gr.Button("Clasificar Imagen")

    # Componente para mostrar la clasificación
    label_output = gr.Label(label="Clasificación")

    classify_button.click(
        classify_image,
        inputs=image_input,
        outputs=label_output
    )

# Lanzar la app en Colab
app.launch(share=True, debug=True)